In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import time
import pandas as pd

# -----------------------------
# 1) پروفایل کامیون
# -----------------------------
df = pd.read_excel("book.xlsx")

# Loading (PERT)
A_LOAD, M_LOAD, B_LOAD = 3, 4.5, 6

# Unloading (PERT)
A_UNLOAD, M_UNLOAD, B_UNLOAD = 1, 2, 3

# travel time (PERT)
A_TRAVEL, M_TRAVEL, B_TRAVEL = 15.5, 16.75, 18

def truck_profile(truck_id):
    unload_time = np.array([])
    loading_time = np.array([])
    total_cycle_time = np.array([])
    travel_time = (total_cycle_time - loading_time - unload_time) / 2
    failure_prob = np.array([])
    failure_time = np.array([])

    return {
        "unload_time": unload_time,
        "travel_time": travel_time,
        "failure_prob": failure_prob,
        "failure_time": failure_time
    }

# -----------------------------
# 2) Monte Carlo
# -----------------------------
def monte_carlo_simulation(data, n_sim=10000):
    start = time.time()
    samples = np.random.choice(data, size=n_sim, replace=True)
    exec_time = time.time() - start
    return samples, exec_time

# -----------------------------
# 3) PERT
# -----------------------------
def pert_params(data):
    a = np.min(data)
    b = np.max(data)
    m = np.mean(data)
    return a, m, b

def pert_simulation(a, m, b, n_sim=10000):
    start = time.time()
    alpha = 1 + 4 * (m - a) / (b - a)
    beta_param = 1 + 4 * (b - m) / (b - a)
    samples = a + (b - a) * np.random.beta(alpha, beta_param, size=n_sim)
    exec_time = time.time() - start
    return samples, exec_time

# -----------------------------
# 4) نمودار
# -----------------------------
def plot_mc_vs_pert(data, title):
    mc, _ = monte_carlo_simulation(data)
    a, m, b = pert_params(data)
    pert, _ = pert_simulation(a, m, b)

    plt.figure()
    plt.hist(mc, bins=30, density=True, alpha=0.5, label="Monte Carlo")
    plt.hist(pert, bins=30, density=True, alpha=0.5, label="PERT")
    plt.legend()
    plt.title(title)
    plt.xlabel("Value")
    plt.ylabel("Probability Density")
    plt.show()

# -----------------------------
# 5) جدول مقایسه
# -----------------------------
def comparison_table(data):
    mc, mc_time = monte_carlo_simulation(data)
    a, m, b = pert_params(data)
    pert, pert_time = pert_simulation(a, m, b)

    table = pd.DataFrame({
        "Method": ["Monte Carlo", "PERT"],
        "Mean": [mc.mean(), pert.mean()],
        "Variance": [mc.var(), pert.var()],
        "Execution Time (s)": [mc_time, pert_time]
    })

    return table

# -----------------------------
# 6) اجرای کامل برای یک کامیون
# -----------------------------
def analyze_truck(truck_id):
    profile = truck_profile(truck_id)

    print("Travel Time Comparison")
    print(comparison_table(profile["travel_time"]))
    plot_mc_vs_pert(profile["travel_time"], "Travel Time: MC vs PERT")

    print("Unload Time Comparison")
    print(comparison_table(profile["unload_time"]))
    plot_mc_vs_pert(profile["unload_time"], "Unload Time: MC vs PERT")

    print("Failure Time Comparison")
    print(comparison_table(profile["failure_time"]))
    plot_mc_vs_pert(profile["failure_time"], "Failure Time: MC vs PERT")


NameError: name 'INSTALL_MAPPING' is not defined